<a href="https://colab.research.google.com/github/nimanumaeh/Digit-Reading/blob/main/CNN_digit_reading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import cv2
import numpy as np
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR


def preprocess_image(image_path):
    # Open the image using PIL and convert to grayscale
    img = Image.open(image_path).convert('L')

    # Resize image to 28x28 pixels
    img = img.resize((28, 28), Image.LANCZOS)

    # Normalize pixel values
    img_array = np.array(img) / 255.0

    # Convert to PyTorch tensor
    img_tensor = transforms.ToTensor()(img_array).float()
    img_tensor = img_tensor.unsqueeze(0)  # Add batch dimension
    return img_tensor

def predict_digit(image_path, model, device):
    model.eval()
    with torch.no_grad():
        img = preprocess_image(image_path)
        img = img.to(device)
        output = model(img)
        pred = output.argmax(dim=1, keepdim=True)
        return pred.item()


class DigitCNN(nn.Module):
    def __init__(self):
        super(DigitCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)

        # Calculate the size of the flattened features after conv layers
        conv_output_size = self._get_conv_output_size((1, 28, 28))

        # Fully connected layers
        self.fc1 = nn.Linear(conv_output_size, 128)
        self.fc2 = nn.Linear(128, 10)

    def _get_conv_output_size(self, input_size):
        # Pass a dummy input through the convolutional layers
        dummy_input = torch.zeros(1, *input_size)
        dummy_output = self._forward_conv(dummy_input)
        return int(torch.numel(dummy_output))

    def _forward_conv(self, x):
        # Forward pass through the convolutional layers
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        return x

    def forward(self, x):
        # Forward pass through the entire network
        x = self._forward_conv(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


# Function to calculate accuracy
def calculate_accuracy(y_pred, y):
    predicted = torch.argmax(y_pred, dim=1)
    correct = (predicted == y).float().sum()
    return correct / y.shape[0]

# Training function with accuracy and periodic validation
def train_and_validate(model, device, train_loader, val_loader, optimizer, epoch, scheduler):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % (len(train_loader) // 10) == 0:  # Reduced frequency of validation
            train_loss = loss.item()
            train_accuracy = calculate_accuracy(output, target).item()
            print(f'Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\t'
                  f'Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.4f}')

    scheduler.step()  # Adjust learning rate
    val_loss, val_accuracy = validate(model, device, val_loader)  # Validate after each epoch
    print(f'Validation Loss: {val_loss:.6f}, Validation Accuracy: {val_accuracy:.4f}')


# Validation function
def validate(model, device, val_loader):
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += F.nll_loss(output, target, reduction='sum').item()
            val_accuracy += calculate_accuracy(output, target)

    val_loss /= len(val_loader.dataset)
    val_accuracy /= len(val_loader)
    return val_loss, val_accuracy

# Main execution block
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = DigitCNN().to(device)

    dataset = MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    learning_rates = 0.001
    batch_size = 32


    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)  # Using multiple workers
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)  # Learning rate scheduler

    for epoch in range(1,4):
        train_and_validate(model, device, train_loader, val_loader, optimizer, epoch, scheduler)

    torch.save(model.state_dict(), 'digit_cnn_model.pth')

Epoch: 1 [0/48000 (0%)]	Train Loss: 2.288598, Train Accuracy: 0.1250
Epoch: 1 [4800/48000 (10%)]	Train Loss: 0.701374, Train Accuracy: 0.8438
Epoch: 1 [9600/48000 (20%)]	Train Loss: 0.313486, Train Accuracy: 0.9062
Epoch: 1 [14400/48000 (30%)]	Train Loss: 0.175344, Train Accuracy: 0.9688
Epoch: 1 [19200/48000 (40%)]	Train Loss: 0.836050, Train Accuracy: 0.8125
Epoch: 1 [24000/48000 (50%)]	Train Loss: 0.190951, Train Accuracy: 0.9062
Epoch: 1 [28800/48000 (60%)]	Train Loss: 0.056992, Train Accuracy: 1.0000
Epoch: 1 [33600/48000 (70%)]	Train Loss: 0.097359, Train Accuracy: 0.9688
Epoch: 1 [38400/48000 (80%)]	Train Loss: 0.135325, Train Accuracy: 0.9375
Epoch: 1 [43200/48000 (90%)]	Train Loss: 0.020698, Train Accuracy: 1.0000
Validation Loss: 0.116093, Validation Accuracy: 0.9653
Epoch: 2 [0/48000 (0%)]	Train Loss: 0.210927, Train Accuracy: 0.9688
Epoch: 2 [4800/48000 (10%)]	Train Loss: 0.004112, Train Accuracy: 1.0000
Epoch: 2 [9600/48000 (20%)]	Train Loss: 0.032940, Train Accuracy: 1.00

In [28]:
import io
from PIL import Image, ImageOps
import torch
from google.colab import files
from torchvision import transforms
from IPython.display import display
import numpy as np

# Assuming DigitCNN and preprocess_image are defined in your main.py

# Load model
model = DigitCNN()  # Create an instance of your model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('digit_cnn_model.pth', map_location=device))  # Load the pre-trained model
model.to(device)

def preprocess_image(img):
    # Convert to grayscale
    img = img.convert('L')
    print("Image converted to grayscale successfully.")

    # Resize the image to 28x28 pixels
    img = img.resize((28, 28), Image.LANCZOS)
    print("Image resized successfully.")

    # Invert colors if necessary (for MNIST images which have white digits on black background)
    img = ImageOps.invert(img)
    print("Image color inversion done successfully.")

    # Convert to numpy array and normalize
    img_array = np.array(img) / 255.0
    print("Image converted to numpy array and normalized successfully.")

    # Convert to PyTorch tensor
    img_tensor = transforms.ToTensor()(img_array).float().unsqueeze(0)
    print("Image converted to PyTorch tensor successfully.")

    return img_tensor

def upload_and_predict():
    uploaded = files.upload()
    for file_name in uploaded.keys():
        img = Image.open(io.BytesIO(uploaded[file_name]))
        print("Image has been opened successfully.")

        # Preprocess the image
        img_tensor = preprocess_image(img).to(device)
        print("Image has been preprocessed successfully.")

        # Predict and display the result
        model.eval()
        with torch.no_grad():
            output = model(img_tensor)
            pred = output.argmax(dim=1, keepdim=True).item()
            print(f"Predicted Digit: {pred}")

        # Display the processed image (optional, for verification)
        img_processed = preprocess_image(img)
        display_img = Image.fromarray((img_processed.squeeze(0).cpu().numpy()[0] * 255).astype(np.uint8), mode='L')
        display(display_img)
        print("Image displayed successfully.")

# Run this function to upload an image and get a prediction
upload_and_predict()


Saving sample_image.png to sample_image.png
Image has been opened successfully.
Image converted to grayscale successfully.
Image resized successfully.
Image color inversion done successfully.
Image converted to numpy array and normalized successfully.
Image converted to PyTorch tensor successfully.
Image has been preprocessed successfully.
Predicted Digit: 7
Image converted to grayscale successfully.
Image resized successfully.
Image color inversion done successfully.
Image converted to numpy array and normalized successfully.
Image converted to PyTorch tensor successfully.


Image displayed successfully.
